In [ ]:

import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
from datetime import datetime
import os

def load_master_data_safely():
    """Safely load master data and check columns"""
    print("Loading master dataset safely...")
    
    try:
        # Try loading with coordinates first
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
            print(f" Master data with coordinates loaded: {master_data.shape}")
        else:
            # Load original master data
            master_data = pl.read_parquet("data/master_transactions.parquet")
            print(f"Master data loaded: {master_data.shape}")
            
            # Add coordinates if needed
            print("Adding coordinates...")
            stores_original = pd.read_csv(r'C:\Users\Alan\Downloads\BDM\BDM_raw_data\stores.csv')
            stores_coords = pl.from_pandas(stores_original).select([
                'Store ID', 'Latitude', 'Longitude'
            ])
            
            master_data = master_data.join(stores_coords, on='Store ID', how='left')
            master_data.write_parquet("data/master_transactions_with_coords.parquet")
            print(" Coordinates added and saved!")
        
        # Verify key columns exist
        required_columns = ['Line_Total_USD', 'Unit_Price_USD', 'Quantity', 'Country', 'Transaction Type']
        missing_columns = [col for col in required_columns if col not in master_data.columns]
        
        if missing_columns:
            print(f" Missing columns: {missing_columns}")
            return None
        
        print(" All required columns found!")
        return master_data
        
    except Exception as e:
        print(f" Errror loading master data: {e}")
        return None

def prepare_optimized_geographic_data(master_data):
    """Prepare optimized data for geographic analysis using SIMPLE discount calculation"""
    print(" Preparing optimized geographic analysis data...")
    
    # Filter for sales only (exclude returns)
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")
    
    # Verify we have the required columns
    print(" Verifying required columns in sales data...")
    required_cols = ['Line_Total_USD', 'Unit_Price_USD', 'Quantity', 'Country', 'Date']
    for col in required_cols:
        if col not in sales_data.columns:
            print(f" Missing column: {col}")
            return None
        else:
            print(f" Found column: {col}")
    
    # Add time-based columns
    sales_data = sales_data.with_columns([
        pl.col("Date").dt.strftime("%Y-%m").alias("Year_Month"),
        pl.col("Date").dt.strftime("%Y-W%U").alias("Year_Week"),
        pl.col("Date").dt.weekday().alias("Day_of_Week_Num"),
        pl.col("Date").dt.strftime("%A").alias("Day_of_Week_Name")
    ])
    
    print(" Creating optimized data aggregations with SIMPLE discount calculation...")
    
    # 1. Monthly trends - ALL data (cumulative)
    print("Creating monthly trends...")
    monthly_trends = sales_data.group_by("Year_Month").agg([
        pl.col("Line_Total_USD").sum().alias("Monthly_Sales_USD"),
        pl.col("Quantity").sum().alias("Monthly_Quantity"),
        pl.col("Line_Total_USD").count().alias("Monthly_Transactions")
    ]).sort("Year_Month")
    
    # 2. Weekly trends - ALL data (cumulative)
    print("Creating weekly trends...")
    weekly_trends = sales_data.group_by("Year_Week").agg([
        pl.col("Line_Total_USD").sum().alias("Weekly_Sales_USD"),
        pl.col("Quantity").sum().alias("Weekly_Quantity"),
        pl.col("Line_Total_USD").count().alias("Weekly_Transactions")
    ]).sort("Year_Week")
    
    # 3. Monthly trends by country - ALL countries
    print("Creating monthly country trends...")
    monthly_country_trends = sales_data.group_by(["Country", "Year_Month"]).agg([
        pl.col("Line_Total_USD").sum().alias("Monthly_Sales_USD"),
        pl.col("Quantity").sum().alias("Monthly_Quantity")
    ]).sort(["Country", "Year_Month"])
    
    # 4. COMBINED Country-wise analysis using SIMPLE discount calculation
    print("Creating country combined analysis...")
    country_combined = sales_data.group_by("Country").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Store ID").n_unique().alias("Number_of_Stores"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        # SIMPLE discount calculation - Unit_Price_USD * Quantity = Total without discount
        (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        # Calculate average realization
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        # Calculate simple average discount percentage
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) / 
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ]).sort("Total_Sales_USD", descending=True)


    # 5. COMBINED City-wise analysis - FIXED to use store-level data
    print("Creating city combined analysis...")
    # First get store-level aggregated data with discount calculation
    store_level_with_discount = sales_data.filter(
        pl.col("Store ID").is_not_null() &
        pl.col("Store Name").is_not_null() &
        pl.col("Country").is_not_null() &
        pl.col("City").is_not_null()
    ).group_by([
        "Store ID"
    ]).agg([
        pl.col("Store Name").first().alias("Store_Name"),
        pl.col("Country").first().alias("Country"),
        pl.col("City").first().alias("City"),
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) / 
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ])
    
    # Then aggregate by city using store-level data
    city_combined = store_level_with_discount.group_by(["Country", "City"]).agg([
        pl.col("Total_Sales_USD").sum().alias("Total_Sales_USD"),
        pl.col("Total_Quantity").sum().alias("Total_Quantity"),
        pl.col("Total_Transactions").sum().alias("Total_Transactions"),
        pl.col("Store ID").count().alias("Number_of_Stores"),
        pl.col("Unique_Customers").sum().alias("Unique_Customers"),
        pl.col("Total_Without_Discount_USD").sum().alias("Total_Without_Discount_USD")
    ]).with_columns([
        (pl.col("Total_Sales_USD") / pl.col("Total_Quantity")).alias("Avg_Realization_USD"),
        ((pl.col("Total_Without_Discount_USD") - pl.col("Total_Sales_USD")) / 
         pl.col("Total_Without_Discount_USD") * 100).alias("Avg_Discount_Percent")
    ]).sort("Total_Sales_USD", descending=True)

    
    # 6. Store-level for heatmap - ALL stores - FIXED
    print("Creating store heatmap data...")
    store_heatmap = sales_data.filter(
        pl.col("Store ID").is_not_null() &
        pl.col("Store Name").is_not_null() &
        pl.col("Country").is_not_null() &
        pl.col("City").is_not_null()
    ).group_by([
        "Store ID"
    ]).agg([
        pl.col("Store Name").first().alias("Store Name"),
        pl.col("Country").first().alias("Country"),
        pl.col("City").first().alias("City"),
        pl.col("Latitude").first().alias("Latitude"),
        pl.col("Longitude").first().alias("Longitude"),
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions")
    ]).sort("Total_Sales_USD", descending=True)

    
    # 7. Day of week analysis - Sales and Quantity separately
    print("Creating day of week analysis...")
    day_of_week_sales = sales_data.group_by(["Day_of_Week_Num", "Day_of_Week_Name"]).agg([
        pl.col("Line_Total_USD").sum().alias("Total_Sales_USD")
    ]).sort("Day_of_Week_Num")
    
    day_of_week_quantity = sales_data.group_by(["Day_of_Week_Num", "Day_of_Week_Name"]).agg([
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).sort("Day_of_Week_Num")
    
    print(" Optimized geographic data prepared with SIMPLE discount calculation")
    print(" Discount calculation:")
    print("   • Total Without Discount = Unit_Price_USD × Quantity")
    print("   • Total With Discount = Line_Total_USD")
    print("   • Discount % = (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100")
    
    return {
        "sales_data": sales_data,
        "monthly_trends": monthly_trends,
        "weekly_trends": weekly_trends,
        "monthly_country_trends": monthly_country_trends,
        "country_combined": country_combined,  # Combined data
        "city_combined": city_combined,        # Combined data
        "store_heatmap": store_heatmap,
        "day_of_week_sales": day_of_week_sales,
        "day_of_week_quantity": day_of_week_quantity
    }


def create_optimized_geographic_dashboard(data_dict):
    """Create optimized dashboard with combined visualizations"""
    print(" Creating optimized geographic dashboard...")
    
    # Convert all data to pandas for plotting
    monthly_trends_df = data_dict["monthly_trends"].to_pandas()
    weekly_trends_df = data_dict["weekly_trends"].to_pandas()
    monthly_country_df = data_dict["monthly_country_trends"].to_pandas()
    country_df = data_dict["country_combined"].to_pandas()  # Combined data
    city_df = data_dict["city_combined"].to_pandas()        # Combined data
    print(city_df.shape)
    store_df = data_dict["store_heatmap"].to_pandas()
    day_sales_df = data_dict["day_of_week_sales"].to_pandas()
    day_qty_df = data_dict["day_of_week_quantity"].to_pandas()
    
    # Create HTML content
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Optimized Geographic Analysis Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .column-info { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
        </style>
    </head>
    <body>
        <div class="dashboard-title"> Optimized Geographic Analysis Dashboard</div>
        <div class="column-info">
            <strong> Simple Discount Calculation:</strong><br>
            • <strong>Total Without Discount</strong>: Unit Price × Quantity<br>
            • <strong>Total With Discount</strong>: Line_Total_USD<br>
            • <strong>Discount %</strong>: (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100
        </div>
    """
    
    # 1. Monthly Cumulative Trends
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(
        x=monthly_trends_df['Year_Month'],
        y=monthly_trends_df['Monthly_Sales_USD'],
        mode='lines+markers',
        name='Monthly Sales (USD)',
        line=dict(color='#1f77b4', width=3),
        marker=dict(size=8)
    ))
    fig1.update_layout(
        title=" Monthly Sales Trends (Cumulative)",
        xaxis_title="Month",
        yaxis_title="Sales (USD)",
        height=500,
        template="plotly_white"
    )
    fig1.update_xaxes(tickangle=45)
    
    # 2. Weekly Cumulative Trends
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(
        x=weekly_trends_df['Year_Week'],
        y=weekly_trends_df['Weekly_Sales_USD'],
        mode='lines+markers',
        name='Weekly Sales (USD)',
        line=dict(color='#ff7f0e', width=2),
        marker=dict(size=6)
    ))
    fig2.update_layout(
        title=" Weekly Sales Trends (Cumulative)",
        xaxis_title="Week",
        yaxis_title="Sales (USD)",
        height=500,
        template="plotly_white"
    )
    fig2.update_xaxes(tickangle=45)
    
    # 3. Monthly Trends by Country (Line Graph)
    fig3 = px.line(
        monthly_country_df,
        x="Year_Month",
        y="Monthly_Sales_USD",
        color="Country",
        title=" Monthly Sales Trends by Country",
        markers=True,
        height=600
    )
    fig3.update_xaxes(tickangle=45)
    fig3.update_layout(template="plotly_white")
    
    # 4. COMBINED Country Analysis - Sales & Quantity (bars) + Avg Realization (line)
    fig4 = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add Sales bars
    fig4.add_trace(
        go.Bar(
            x=country_df['Country'],
            y=country_df['Total_Sales_USD'],
            name='Total Sales (Line_Total_USD)',
            marker_color='lightblue',
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add Quantity bars
    fig4.add_trace(
        go.Bar(
            x=country_df['Country'],
            y=country_df['Total_Quantity'],
            name='Total Quantity',
            marker_color='lightgreen',
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add Average Realization line
    fig4.add_trace(
        go.Scatter(
            x=country_df['Country'],
            y=country_df['Avg_Realization_USD'],
            mode='lines+markers',
            name='Avg Realization (Line_Total_USD/Quantity)',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    fig4.update_xaxes(title_text="Country", tickangle=45)
    fig4.update_yaxes(title_text="Sales (USD) / Quantity", secondary_y=False)
    fig4.update_yaxes(title_text="Average Realization (USD)", secondary_y=True)
    fig4.update_layout(
        title_text="  Country Analysis: Sales, Quantity & Avg Realization",
        height=600,
        template="plotly_white"
    )
    
    # 5. COMBINED City Analysis - Sales & Quantity (bars) + Avg Realization (line)
    city_df['City_Country'] = city_df['City'] + ', ' + city_df['Country']
    fig5 = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add Sales bars
    fig5.add_trace(
        go.Bar(
            x=city_df['City_Country'],
            y=city_df['Total_Sales_USD'],
            name='Total Sales (Line_Total_USD)',
            marker_color='lightcoral',
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add Quantity bars
    fig5.add_trace(
        go.Bar(
            x=city_df['City_Country'],
            y=city_df['Total_Quantity'],
            name='Total Quantity',
            marker_color='lightpink',
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add Average Realization line
    fig5.add_trace(
        go.Scatter(
            x=city_df['City_Country'],
            y=city_df['Avg_Realization_USD'],
            mode='lines+markers',
            name='Avg Realization (Line_Total_USD/Quantity)',
            line=dict(color='orange', width=3),
            marker=dict(size=6),
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    fig5.update_xaxes(title_text="City, Country", tickangle=45)
    fig5.update_yaxes(title_text="Sales (USD) / Quantity", secondary_y=False)
    fig5.update_yaxes(title_text="Average Realization (USD)", secondary_y=True)
    fig5.update_layout(
        title_text="  City Analysis: Sales, Quantity & Avg Realization",
        height=700,
        template="plotly_white"
    )
    
    # 6. Store Heatmap (Scatter Map)
    store_clean = store_df.dropna(subset=['Latitude', 'Longitude'])
    fig6 = px.scatter_mapbox(
        store_clean,
        lat="Latitude",
        lon="Longitude",
        size="Total_Sales_USD",
        color="Total_Sales_USD",
        hover_name="Store Name",
        hover_data={
            "Country": True,
            "City": True,
            "Total_Sales_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Total_Transactions": ":,"
        },
        color_continuous_scale="Plasma",
        size_max=50,
        zoom=1,
        title=" Store Sales Heatmap",
        height=700
    )
    fig6.update_layout(mapbox_style="open-street-map", template="plotly_white")
    
    # 7. Day of Week - Sales Analysis
    fig7 = go.Figure()
    fig7.add_trace(go.Bar(
        x=day_sales_df['Day_of_Week_Name'],
        y=day_sales_df['Total_Sales_USD'],
        name='Sales by Day',
        marker_color='skyblue'
    ))
    fig7.update_layout(
        title=" Sales by Day of Week",
        xaxis_title="Day of Week",
        yaxis_title="Total Sales (USD)",
        height=500,
        template="plotly_white"
    )
    
    # 8. Day of Week - Quantity Analysis
    fig8 = go.Figure()
    fig8.add_trace(go.Bar(
        x=day_qty_df['Day_of_Week_Name'],
        y=day_qty_df['Total_Quantity'],
        name='Quantity by Day',
        marker_color='lightseagreen'
    ))
    fig8.update_layout(
        title=" Quantity by Day of Week",
        xaxis_title="Day of Week",
        yaxis_title="Total Quantity",
        height=500,
        template="plotly_white"
    )
    
    # 9. SIMPLE Discount vs Sales by Country
    fig9 = px.scatter(
        country_df,
        x="Avg_Discount_Percent",
        y="Total_Sales_USD",
        text="Country",
        title=" Average Discount % vs Total Sales by Country<br><sub>Simple Calculation: (Unit Price × Quantity - Line_Total_USD) ÷ (Unit Price × Quantity) × 100</sub>",
        height=600
    )
    fig9.update_traces(textposition="top center")
    fig9.update_layout(
        xaxis_title="Average Discount (%)",
        yaxis_title="Total Sales (Line_Total_USD)",
        template="plotly_white"
    )
    
    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9]
    titles = [
        "Monthly Sales Trends (Cumulative)",
        "Weekly Sales Trends (Cumulative)", 
        "Monthly Sales Trends by Country",
        "Combined Country Analysis: Sales, Quantity & Avg Realization",
        "Combined City Analysis: Sales, Quantity & Avg Realization",
        "Store Sales Heatmap",
        "Sales by Day of Week",
        "Quantity by Day of Week",
        "Simple Discount vs Sales by Country"
    ]
    
    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """
    
    html_content += """
    <script>
    """
    
    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """
    
    html_content += """
    </script>
    </body>
    </html>
    """
    
    return html_content

def save_optimized_data_files(data_dict):
    """Save optimized combined data files"""
    print(" Saving optimized combined data files...")
    
    os.makedirs("geographic_analysis/data", exist_ok=True)
    
    # Save optimized datasets
    data_dict["monthly_trends"].write_csv("geographic_analysis/data/monthly_trends.csv")
    data_dict["weekly_trends"].write_csv("geographic_analysis/data/weekly_trends.csv")
    data_dict["monthly_country_trends"].write_csv("geographic_analysis/data/monthly_country_trends.csv")
    data_dict["country_combined"].write_csv("geographic_analysis/data/country_combined_analysis.csv")  # Combined
    data_dict["city_combined"].write_csv("geographic_analysis/data/city_combined_analysis.csv")        # Combined
    data_dict["store_heatmap"].write_csv("geographic_analysis/data/store_heatmap.csv")
    data_dict["day_of_week_sales"].write_csv("geographic_analysis/data/day_of_week_sales.csv")
    data_dict["day_of_week_quantity"].write_csv("geographic_analysis/data/day_of_week_quantity.csv")
    
    print(" Optimized data files saved with SIMPLE discount calculation!")

def run_optimized_geographic_analysis():
    """Run the optimized geographic analysis with simple discount calculation"""
    print(" Starting Optimized Geographic Analysis with SIMPLE Discount Calculation...")
    
    # Load data safely
    master_data = load_master_data_safely()
    if master_data is None:
        print(" Failed to load master data!")
        return None
    
    # Print column info for verification
    print(f"\n Master data shape: {master_data.shape}")
    print(" Key columns verified:")
    key_cols = ['Line_Total_USD', 'Unit Price', 'Quantity', 'Country', 'Transaction Type']
    for col in key_cols:
        if col in master_data.columns:
            print(f"    {col}")
        else:
            print(f"    {col} - MISSING!")
    
    # Prepare optimized data
    data_dict = prepare_optimized_geographic_data(master_data)
    if data_dict is None:
        print(" Failed to prepare data!")
        return None
    
    # Save optimized data files
    save_optimized_data_files(data_dict)
    
    # Create optimized dashboard
    html_content = create_optimized_geographic_dashboard(data_dict)
    
    # Create directory and save HTML
    os.makedirs("geographic_analysis", exist_ok=True)
    
    with open("geographic_analysis/optimized_geographic_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)
    
    # Print summary
    country_df = data_dict["country_combined"].to_pandas()
    city_df = data_dict["city_combined"].to_pandas()
    
    print("\n OPTIMIZED GEOGRAPHIC ANALYSIS COMPLETE!")
    print(f" Total Global Sales (Line_Total_USD): ${country_df['Total_Sales_USD'].sum():,.0f}")
    print(f" Total Quantity Sold: {country_df['Total_Quantity'].sum():,}")
    print(f" Countries Analyzed: {len(country_df)}")
    print(f" Cities Analyzed: {len(city_df)}")
    print(f" Global Avg Realization: ${country_df['Total_Sales_USD'].sum() / country_df['Total_Quantity'].sum():.2f}")
    print(f" Global Avg Discount: {((country_df['Total_Without_Discount_USD'].sum() - country_df['Total_Sales_USD'].sum()) / country_df['Total_Without_Discount_USD'].sum() * 100):.2f}%")
    
    print("\n Dashboard Created with SIMPLE Discount Calculation:")
    print("    geographic_analysis/optimized_geographic_dashboard.html")
    print("    geographic_analysis/data/ (Optimized CSV files)")
    
    print("\n  Discount Calculation Applied:")
    print("    Total Without Discount = Unit Price USD × Quantity")
    print("    Total With Discount = Line_Total_USD")
    print("    Discount % = (Total Without Discount - Total With Discount) ÷ Total Without Discount × 100")
    print("    Using actual column names from your data!")
    
    return data_dict

# Run the optimized analysis with simple discount calculation
results = run_optimized_geographic_analysis()


 Starting Optimized Geographic Analysis with SIMPLE Discount Calculation...
Loading master dataset safely...
 Master data with coordinates loaded: (6416827, 45)
 All required columns found!

 Master data shape: (6416827, 45)
 Key columns verified:
    Line_Total_USD
    Unit Price
    Quantity
    Country
    Transaction Type
 Preparing optimized geographic analysis data...
Sales data shape: (6077200, 45)
 Verifying required columns in sales data...
 Found column: Line_Total_USD
 Found column: Unit_Price_USD
 Found column: Quantity
 Found column: Country
 Found column: Date
 Creating optimized data aggregations with SIMPLE discount calculation...
Creating monthly trends...
Creating weekly trends...
Creating monthly country trends...
Creating country combined analysis...
Creating city combined analysis...
Creating store heatmap data...
Creating day of week analysis...
 Optimized geographic data prepared with SIMPLE discount calculation
 Discount calculation:
   • Total Without Discount 

Prcing Analysis


In [8]:
import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import numpy as np
from datetime import datetime
import os

def load_master_data_for_pricing():
    """Load master data for pricing analysis"""
    print("Loading master data for pricing analysis...")
    
    try:
        if os.path.exists("data/master_transactions_with_coords.parquet"):
            master_data = pl.read_parquet("data/master_transactions_with_coords.parquet")
        else:
            master_data = pl.read_parquet("data/master_transactions.parquet")
        
        print(f"Master data loaded: {master_data.shape}")
        return master_data
        
    except Exception as e:
        print(f"Error loading master data: {e}")
        return None

def prepare_pricing_analysis_data(master_data):
    """Prepare comprehensive pricing and discount strategy analysis data"""
    print("Preparing pricing and discount strategy analysis data...")
    
    # Filter for sales only
    sales_data = master_data.filter(pl.col("Transaction Type") == "Sale")
    print(f"Sales data shape: {sales_data.shape}")
    
    # Add discount calculations
    sales_data = sales_data.with_columns([
        # Calculate discount amount and percentage
        (pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")).alias("Discount_Amount_USD"),
        ((pl.col("Unit_Price_USD") * pl.col("Quantity") - pl.col("Line_Total_USD")) / 
         (pl.col("Unit_Price_USD") * pl.col("Quantity")) * 100).alias("Discount_Percent"),
        
        # Add time dimensions
        pl.col("Date").dt.strftime("%Y-%m").alias("Year_Month"),
        pl.col("Date").dt.quarter().alias("Quarter"),
        
        # Product identifier
        (pl.col("Product ID").cast(pl.Utf8) + "_" + pl.col("Size") + "_" + pl.col("Color")).alias("Product_SKU")
    ])
    
    print("Creating product-level aggregations...")
    
    # 1. Product Performance Matrix Data
    product_performance = sales_data.group_by([
        "Product ID", "Category", "Sub Category", "Description EN"
    ]).agg([
        # Revenue metrics
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        
        # Price metrics
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),
        
        # Discount metrics - WEIGHTED AVERAGE
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Discount_Amount_USD").sum().alias("Total_Discount_USD"),
        
        # Customer metrics
        pl.col("Customer ID").n_unique().alias("Unique_Customers"),
        pl.col("Store ID").n_unique().alias("Stores_Available")
    ]).with_columns([
        # Calculate additional metrics
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit"),
        (pl.col("Total_Discount_USD") / pl.col("Total_Quantity")).alias("Discount_Per_Unit"),
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer")
    ])
    
    # Add percentile ranks for matrix positioning
    product_performance = product_performance.with_columns([
        pl.col("Total_Revenue_USD").rank(method="average").alias("Revenue_Rank"),
        pl.col("Avg_Discount_Percent").rank(method="average").alias("Discount_Rank")
    ]).with_columns([
        # Convert to percentile (0-1 scale)
        (pl.col("Revenue_Rank") / pl.col("Revenue_Rank").max()).alias("Revenue_Percentile_Rank"),
        (pl.col("Discount_Rank") / pl.col("Discount_Rank").max()).alias("Discount_Percentile_Rank")
    ]).sort("Total_Revenue_USD", descending=True)
    
    print("Creating price elasticity analysis...")
    
    # 2. Price Elasticity Analysis (Product + Size + Color level)
    price_elasticity = sales_data.group_by([
        "Product ID", "Size", "Color", "Category", "Sub Category"
    ]).agg([
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),  # Final price after discount
        pl.col("Quantity").sum().alias("Total_Quantity_Sold"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions")
    ]).filter(
        # Filter for products with meaningful sales
        (pl.col("Total_Quantity_Sold") >= 10) & 
        (pl.col("Total_Transactions") >= 5)
    ).sort("Total_Revenue_USD", descending=True)

    print("Creating discount effectiveness analysis...")
    
    # 3. Discount Effectiveness by Category
    discount_effectiveness = sales_data.group_by([
        "Category", "Sub Category"
    ]).agg([
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit"),
        (pl.col("Total_Revenue_USD") / pl.col("Unique_Customers")).alias("Revenue_Per_Customer")
    ]).sort("Total_Revenue_USD", descending=True)
    
    print("Creating discount bucket analysis...")
    
    # 4. Discount Bucket Analysis
    discount_buckets = sales_data.with_columns([
        pl.when(pl.col("Discount_Percent") <= 5)
        .then(pl.lit("0-5%"))
        .when(pl.col("Discount_Percent") <= 15)
        .then(pl.lit("6-15%"))
        .when(pl.col("Discount_Percent") <= 25)
        .then(pl.lit("16-25%"))
        .when(pl.col("Discount_Percent") <= 35)
        .then(pl.lit("26-35%"))
        .when(pl.col("Discount_Percent") <= 50)
        .then(pl.lit("36-50%"))
        .otherwise(pl.lit("50%+"))
        .alias("Discount_Bucket")
    ]).group_by("Discount_Bucket").agg([
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity"),
        pl.col("Line_Total_USD").count().alias("Total_Transactions"),
        pl.col("Customer ID").n_unique().alias("Unique_Customers")
    ]).with_columns([
        (pl.col("Total_Revenue_USD") / pl.col("Total_Quantity")).alias("Revenue_Per_Unit")
    ])
    
    print("Creating temporal pricing trends...")
    
    # 5. Monthly Pricing Trends
    monthly_pricing = sales_data.group_by("Year_Month").agg([
        (pl.col("Line_Total_USD").sum() / pl.col("Quantity").sum()).alias("Avg_Unit_Price_USD"),
        (pl.col("Discount_Amount_USD").sum() / (pl.col("Unit_Price_USD") * pl.col("Quantity")).sum() * 100).alias("Avg_Discount_Percent"),
        pl.col("Line_Total_USD").sum().alias("Total_Revenue_USD"),
        pl.col("Quantity").sum().alias("Total_Quantity")
    ]).sort("Year_Month")
    
    print("Pricing analysis data prepared successfully!")
    
    return {
        "sales_data": sales_data,
        "product_performance": product_performance,
        "price_elasticity": price_elasticity,
        "discount_effectiveness": discount_effectiveness,
        "discount_buckets": discount_buckets,
        "monthly_pricing": monthly_pricing
    }


def create_pricing_strategy_dashboard(data_dict):
    """Create comprehensive pricing and discount strategy dashboard"""
    print("Creating Pricing and Discount Strategy Dashboard...")
    
    # Convert to pandas for plotting
    product_perf_df = data_dict["product_performance"].to_pandas()
    price_elasticity_df = data_dict["price_elasticity"].to_pandas()
    discount_eff_df = data_dict["discount_effectiveness"].to_pandas()
    discount_buckets_df = data_dict["discount_buckets"].to_pandas()
    monthly_pricing_df = data_dict["monthly_pricing"].to_pandas()
    
    # Create HTML structure
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Pricing and Discount Strategy Optimization Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            .chart-container { background-color: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .chart-title { font-size: 24px; font-weight: bold; text-align: center; margin-bottom: 20px; color: #333; }
            .dashboard-title { font-size: 36px; font-weight: bold; text-align: center; margin-bottom: 30px; color: #2c3e50; }
            .insights-box { background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 8px; font-size: 14px; }
            .metric-highlight { background-color: #fff3cd; padding: 10px; margin: 5px 0; border-radius: 5px; font-weight: bold; }
        </style>
    </head>
    <body>
        <div class="dashboard-title">Pricing and Discount Strategy Optimization</div>
        
        <div class="insights-box">
            <strong>Key Insights:</strong><br>
            • <strong>Product Performance Matrix</strong>: Shows relationship between discount levels and revenue performance<br>
            • <strong>Price Elasticity</strong>: Analyzes how price changes affect quantity sold<br>
            • <strong>Discount Effectiveness</strong>: Measures ROI of discount strategies across categories<br>
            • <strong>Optimal Pricing</strong>: Identifies sweet spots for maximum profitability
        </div>
    """
    
    # 1. Product Performance Matrix (inspired by your chart)
    fig1 = px.scatter(
        product_perf_df,
        x="Revenue_Percentile_Rank",
        y="Discount_Percentile_Rank", 
        size="Total_Revenue_USD",
        color="Total_Quantity",
        hover_name="Description EN",
        hover_data={
            "Category": True,
            "Sub Category": True,
            "Total_Revenue_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Avg_Discount_Percent": ":.1f%",
            "Avg_Unit_Price_USD": ":.2f"
        },
        title="Product Performance Matrix: Revenue vs Discount Strategy",
        labels={
            "Revenue_Percentile_Rank": "Revenue Percentile Rank",
            "Discount_Percentile_Rank": "Discount Percentile Rank",
            "Total_Quantity": "Total Quantity"
        },
        color_continuous_scale="Viridis",
        height=700
    )
    
    # Add quadrant lines
    fig1.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.7)
    fig1.add_vline(x=0.5, line_dash="dash", line_color="gray", opacity=0.7)
    
    # Add quadrant labels
    fig1.add_annotation(x=0.25, y=0.75, text="High Discount<br>Low Revenue", 
                       showarrow=False, font=dict(size=14, color="red"))
    fig1.add_annotation(x=0.75, y=0.75, text="High Discount<br>High Revenue", 
                       showarrow=False, font=dict(size=14, color="orange"))
    fig1.add_annotation(x=0.25, y=0.25, text="Low Discount<br>Low Revenue", 
                       showarrow=False, font=dict(size=14, color="gray"))
    fig1.add_annotation(x=0.75, y=0.25, text="Low Discount<br>High Revenue", 
                       showarrow=False, font=dict(size=14, color="green"))
    
    fig1.update_layout(template="plotly_white")
    
    # 2. Product Price Elasticity (inspired by your chart)
    fig2 = px.scatter(
        price_elasticity_df,
        x="Avg_Unit_Price_USD",
        y="Total_Quantity_Sold",
        size="Total_Revenue_USD",
        color="Avg_Discount_Percent",
        hover_name="Product ID",
        hover_data={
            "Category": True,
            "Sub Category": True,
            "Size": True,
            "Color": True,
            "Total_Revenue_USD": ":,.0f",
            "Avg_Discount_Percent": ":.1f%"
        },
        title="Product Price Elasticity Analysis",
        labels={
            "Avg_Unit_Price_USD": "Average Unit Price (USD)",
            "Total_Quantity_Sold": "Total Quantity Sold",
            "Avg_Discount_Percent": "Avg Discount %"
        },
        color_continuous_scale="RdYlBu_r",
        height=700,
        log_y=True  # Log scale for better visualization
    )
    fig2.update_layout(template="plotly_white")
    
    # 3. Discount Effectiveness by Category
    fig3 = px.scatter(
        discount_eff_df,
        x="Avg_Discount_Percent",
        y="Revenue_Per_Unit",
        size="Total_Revenue_USD",
        color="Category",
        hover_name="Sub Category",
        hover_data={
            "Total_Revenue_USD": ":,.0f",
            "Total_Quantity": ":,",
            "Unique_Customers": ":,",
            "Avg_Unit_Price_USD": ":.2f"
        },
        title="Discount Effectiveness by Category",
        labels={
            "Avg_Discount_Percent": "Average Discount Percentage (%)",
            "Revenue_Per_Unit": "Revenue Per Unit (USD)"
        },
        height=600
    )
    fig3.update_layout(template="plotly_white")
    
    # 4. Discount Bucket Analysis
    discount_buckets_df['Discount_Order'] = discount_buckets_df['Discount_Bucket'].map({
        '0-5%': 1, '6-15%': 2, '16-25%': 3, '26-35%': 4, '36-50%': 5, '50%+': 6
    })
    discount_buckets_df = discount_buckets_df.sort_values('Discount_Order')
    
    fig4 = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add revenue bars
    fig4.add_trace(
        go.Bar(
            x=discount_buckets_df['Discount_Bucket'],
            y=discount_buckets_df['Total_Revenue_USD'],
            name='Total Revenue (USD)',
            marker_color='lightblue',
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add revenue per unit line
    fig4.add_trace(
        go.Scatter(
            x=discount_buckets_df['Discount_Bucket'],
            y=discount_buckets_df['Revenue_Per_Unit'],
            mode='lines+markers',
            name='Revenue Per Unit (USD)',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    fig4.update_xaxes(title_text="Discount Bucket")
    fig4.update_yaxes(title_text="Total Revenue (USD)", secondary_y=False)
    fig4.update_yaxes(title_text="Revenue Per Unit (USD)", secondary_y=True)
    fig4.update_layout(
        title_text="Revenue Performance by Discount Bucket",
        height=600,
        template="plotly_white"
    )
    
    # 5. Monthly Pricing Trends
    fig5 = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add average price line
    fig5.add_trace(
        go.Scatter(
            x=monthly_pricing_df['Year_Month'],
            y=monthly_pricing_df['Avg_Unit_Price_USD'],
            mode='lines+markers',
            name='Avg Unit Price (USD)',
            line=dict(color='blue', width=3),
            yaxis='y'
        ),
        secondary_y=False,
    )
    
    # Add discount percentage line
    fig5.add_trace(
        go.Scatter(
            x=monthly_pricing_df['Year_Month'],
            y=monthly_pricing_df['Avg_Discount_Percent'],
            mode='lines+markers',
            name='Avg Discount %',
            line=dict(color='red', width=3),
            yaxis='y2'
        ),
        secondary_y=True,
    )
    
    fig5.update_xaxes(title_text="Month", tickangle=45)
    fig5.update_yaxes(title_text="Average Unit Price (USD)", secondary_y=False)
    fig5.update_yaxes(title_text="Average Discount (%)", secondary_y=True)
    fig5.update_layout(
        title_text="Monthly Pricing and Discount Trends",
        height=600,
        template="plotly_white"
    )
    
    # 6. Category Performance Heatmap
    category_pivot = discount_eff_df.pivot_table(
        values='Revenue_Per_Unit', 
        index='Category', 
        columns='Sub Category', 
        fill_value=0
    )
    
    fig6 = px.imshow(
        category_pivot.values,
        labels=dict(x="Sub Category", y="Category", color="Revenue Per Unit"),
        x=category_pivot.columns,
        y=category_pivot.index,
        title="Revenue Per Unit Heatmap by Category & Sub Category",
        color_continuous_scale="RdYlGn",
        height=600
    )
    fig6.update_layout(template="plotly_white")
    
    # 7. Price vs Quantity Correlation by Category
    fig7 = px.scatter(
        price_elasticity_df,
        x="Avg_Unit_Price_USD",
        y="Total_Quantity_Sold",
        color="Category",
        size="Total_Revenue_USD",
        hover_name="Product ID",
        title="Price vs Quantity Correlation by Category",
        labels={
            "Avg_Unit_Price_USD": "Average Unit Price (USD)",
            "Total_Quantity_Sold": "Total Quantity Sold"
        },
        height=600
    )
    fig7.update_layout(template="plotly_white")
    
    # Convert all figures to HTML
    figures = [fig1, fig2, fig3, fig4, fig5, fig6, fig7]
    titles = [
        "Product Performance Matrix: Revenue vs Discount Strategy",
        "Product Price Elasticity Analysis",
        "Discount Effectiveness by Category",
        "Revenue Performance by Discount Bucket",
        "Monthly Pricing and Discount Trends",
        "Revenue Per Unit Heatmap by Category & Sub Category",
        "Price vs Quantity Correlation by Category"
    ]
    
    for i, (fig, title) in enumerate(zip(figures, titles)):
        html_content += f"""
        <div class="chart-container">
            <div class="chart-title">{title}</div>
            <div id="chart{i+1}"></div>
        </div>
        """
    
    # Add key metrics summary
    total_revenue = product_perf_df['Total_Revenue_USD'].sum()
    avg_discount = product_perf_df['Avg_Discount_Percent'].mean()
    best_performing_category = discount_eff_df.loc[discount_eff_df['Revenue_Per_Unit'].idxmax(), 'Category']
    optimal_discount_bucket = discount_buckets_df.loc[discount_buckets_df['Revenue_Per_Unit'].idxmax(), 'Discount_Bucket']
    
    html_content += f"""
        <div class="insights-box">
            <strong>Key Performance Metrics:</strong><br>
            <div class="metric-highlight">Total Revenue: ${total_revenue:,.0f}</div>
            <div class="metric-highlight">Average Discount: {avg_discount:.1f}%</div>
            <div class="metric-highlight">Best Performing Category: {best_performing_category}</div>
            <div class="metric-highlight">Optimal Discount Range: {optimal_discount_bucket}</div>
        </div>
    """
    
    html_content += """
    <script>
    """
    
    # Add JavaScript for each plot
    for i, fig in enumerate(figures):
        plot_json = fig.to_json()
        html_content += f"""
        var plotData{i+1} = {plot_json};
        Plotly.newPlot('chart{i+1}', plotData{i+1}.data, plotData{i+1}.layout, {{responsive: true}});
        """
    
    html_content += """
    </script>
    </body>
    </html>
    """
    
    return html_content

def save_pricing_analysis_data(data_dict):
    """Save pricing analysis data files"""
    print("Saving pricing analysis data files...")
    
    os.makedirs("pricing_analysis/data", exist_ok=True)
    
    # Save all datasets
    data_dict["product_performance"].write_csv("pricing_analysis/data/product_performance_matrix.csv")
    data_dict["price_elasticity"].write_csv("pricing_analysis/data/price_elasticity_analysis.csv")
    data_dict["discount_effectiveness"].write_csv("pricing_analysis/data/discount_effectiveness.csv")
    data_dict["discount_buckets"].write_csv("pricing_analysis/data/discount_bucket_analysis.csv")
    data_dict["monthly_pricing"].write_csv("pricing_analysis/data/monthly_pricing_trends.csv")
    
    print("Pricing analysis data files saved successfully!")

def run_pricing_strategy_analysis():
    """Run comprehensive pricing and discount strategy analysis"""
    print("Starting Pricing and Discount Strategy Optimization Analysis...")
    
    # Load data
    master_data = load_master_data_for_pricing()
    if master_data is None:
        print("Failed to load master data!")
        return None
    
    # Prepare analysis data
    data_dict = prepare_pricing_analysis_data(master_data)
    if data_dict is None:
        print("Failed to prepare pricing data!")
        return None
    
    # Save data files
    save_pricing_analysis_data(data_dict)
    
    # Create dashboard
    html_content = create_pricing_strategy_dashboard(data_dict)
    
    # Create directory and save HTML
    os.makedirs("pricing_analysis", exist_ok=True)
    
    with open("pricing_analysis/pricing_strategy_dashboard.html", "w", encoding='utf-8') as f:
        f.write(html_content)
    
    # Print summary insights
    product_perf_df = data_dict["product_performance"].to_pandas()
    discount_buckets_df = data_dict["discount_buckets"].to_pandas()
    
    print("\nPRICING STRATEGY ANALYSIS COMPLETE!")
    print(f"Total Revenue Analyzed: ${product_perf_df['Total_Revenue_USD'].sum():,.0f}")
    print(f"Products Analyzed: {len(product_perf_df):,}")
    print(f"Average Discount Rate: {product_perf_df['Avg_Discount_Percent'].mean():.1f}%")
    print(f"Highest Revenue Product: {product_perf_df.iloc[0]['Description EN']}")
    
    # Find optimal discount bucket
    optimal_bucket = discount_buckets_df.loc[discount_buckets_df['Revenue_Per_Unit'].idxmax()]
    print(f"Optimal Discount Range: {optimal_bucket['Discount_Bucket']} (Revenue/Unit: ${optimal_bucket['Revenue_Per_Unit']:.2f})")
    
    print("\nDashboard and Data Files Created:")
    print("   pricing_analysis/pricing_strategy_dashboard.html")
    print("   pricing_analysis/data/ (CSV files for detailed analysis)")
    
    return data_dict

# Run the pricing strategy analysis
results = run_pricing_strategy_analysis()


Starting Pricing and Discount Strategy Optimization Analysis...
Loading master data for pricing analysis...
Master data loaded: (6416827, 45)
Preparing pricing and discount strategy analysis data...
Sales data shape: (6077200, 45)
Creating product-level aggregations...
Creating price elasticity analysis...
Creating discount effectiveness analysis...
Creating discount bucket analysis...
Creating temporal pricing trends...
Pricing analysis data prepared successfully!
Saving pricing analysis data files...
Pricing analysis data files saved successfully!
Creating Pricing and Discount Strategy Dashboard...

PRICING STRATEGY ANALYSIS COMPLETE!
Total Revenue Analyzed: $305,884,837
Products Analyzed: 17,940
Average Discount Rate: 8.9%
Highest Revenue Product: Men'S Blazer Of Light Fabric With Cut In Thin Stripes
Optimal Discount Range: 0-5% (Revenue/Unit: $52.59)

Dashboard and Data Files Created:
   pricing_analysis/pricing_strategy_dashboard.html
   pricing_analysis/data/ (CSV files for detai